In [26]:
%pip install --upgrade --quiet  lark langchain-chroma
%pip -q install langchain-openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## План действий
1. Эксперимент на выборке Worki с небольшими диалогами. Документ - весь диалог. Чанки - рекурсивные с границей по сообщению
    1. ✅ Сфомировать документ из диалога
   
    2. ✅Порезать на чанки
2. Модификация для основного датасета где документ - фрагмент диалога кластеризованный по паузе между сообщениями

3. Подумать над установкой логической связи между фрагментами диалога

In [1]:
import json
from dotenv import load_dotenv
import os
from pathlib import Path
from pprint import pprint

load_dotenv()

dialogs_sm = [x for x in json.loads(Path('data/telegram_chats_worki.json').read_text())]
dialogs_sm = [
  {**obj, "messages": [msg for msg in obj["messages"] if msg['text'] and len(msg["text"]) > 0]}
  for obj in dialogs_sm
]

dialogs_md = json.loads(Path('data/telegram_chats.json').read_text())

In [2]:
def chunk_dialog(dialog_messages, chunk_size):
  chunks = []
  current_chunk = []
  current_length = 0

  for message in dialog_messages:
    message_length = len(message['text'])

    if current_length + message_length > chunk_size:
      if current_chunk:
        chunks.append(current_chunk)
      current_chunk = [message]
      current_length = message_length
    else:
      current_chunk.append(message)
      current_length += message_length
      
  if current_chunk:
    chunks.append(current_chunk)

  return chunks

### Маленький датасет. Документ - вся переписка

In [3]:
import re
from langchain_core.documents import Document
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [7]:
docs = []
for dialog in dialogs_sm: 
  dialog_str = ""
  chunked_dialog =chunk_dialog(dialog['messages'], 500)

  for i, chunk in enumerate(chunked_dialog): 
    chat = ""
    for msg in chunk:
      text = re.sub(r'\s+', ' ', msg['text'].replace('"', '\\"').replace('\n', ' ')).strip()
      chat = '{author}({timestamp}): "{message}"\n'.format(author=msg['sender_id'], timestamp=msg['date'], message=text) + chat
    dialog_str = chat + dialog_str

    ##Проверить на баги потенциально не в том порядке склеивается диалог, некогда разбираться. 
    docs.append(Document(page_content=chat,
      metadata={
        "id": '{0}_{1}'.format(dialog['id'], i),
        "chatId": dialog['id'],
        "contactName": dialog['contact_name'],
        "chatType": dialog['type'],
        "timestampFrom": chunk[0]['date'],
        "timestampTo": chunk[-1]['date'] 
      })
    )
  
  with open('chat_logs/dialog_'+str(dialog['id'])+'.txt', 'w', encoding='utf-8') as f:
    f.write(dialog_str)


print(docs)

[Document(metadata={'id': '7066936566_0', 'chatId': 7066936566, 'contactName': 'Worki Team', 'chatType': 'User', 'timestampFrom': '2024-12-16T23:39:29+00:00', 'timestampTo': '2024-10-03T13:07:00+00:00'}, page_content='7066936566(2024-10-03T13:07:00+00:00): "1. WA, tg рассылки 2. Автообзвон по базе МТС Маркетолога 3. Автообзвон по базе владельцев китайских грузовков (где то была такая) 4."\n7066936566(2024-10-07T08:16:40+00:00): "И ещё отдельный вопрос: может ли у вас как-то цена от объема заказа зависеть? Скажем, у меня есть 10 моделей техники, для каждой из которых надо сделать 5 комплектов по 5 фотографий. Если цена будет по 300 рублей за фотку как раньше - это довольно болезненно получается"\n7066936566(2024-10-09T11:03:31+00:00): "Документы за август в с7"\n7066936566(2024-12-16T23:29:43+00:00): "Hello, myself!"\n7066936566(2024-12-16T23:30:58+00:00): "Hello, myself!"\n7066936566(2024-12-16T23:39:29+00:00): "Hello, myself!"\n'), Document(metadata={'id': '7066936566_1', 'chatId': 70

In [13]:
vectorstore = Chroma.from_documents(docs, OpenAIEmbeddings())

In [22]:
results = vectorstore.similarity_search_with_score(
  "размещение на озоне"
)
for res, score in results:
  print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")



* [SIM=0.381626] 5770047663(2024-08-11T17:16:33+00:00): "Мы можем размещать объявления, только в том городе, где подтверждено дилерство. Но наши объявления так же ранжируются на другие регионы, которые мы настраиваем функционалом авито. Если Вы можете сделать дилерство в Москве, то можем и в Москве размещать. Это можно сделать \"по левому\", но этим лучше не баловаться."
7066936566(2024-08-11T17:16:26+00:00): "В целом, я готов стартовать. Но этот вопрос надо включить в родмэп и продумать. Выбор основного региона, который мы будем качать - это важный момент, странно выбирать ЯО только потому что мы исторически тут работали. Вопрос исключительно в потенциале лидгена"
7066936566(2024-08-11T17:07:44+00:00): "вопрос не только в спросе, но и в предложении) в мск например гораздо больше спроса, но и объявлений там больше. Сможем ли мы получать больше лидов, если выберем основным регоном МСК? статус дилера подтвердить сможем, все документы от представительства у нас есть"
 [{'chatId': 57700476

In [39]:
with open('output_file', 'w', encoding='utf-8') as f:
  json.dump(docs_json, f, ensure_ascii=False, indent=4)

In [6]:
vectorstore.get()

{'ids': ['584ad645-0f16-4c47-b179-0204bf0e2791',
  'c0330c51-c01f-4c57-a242-a5dc2bf7bf87',
  '3198956b-15ab-481e-b1af-e5fcaef006fc',
  '945dddac-6bd7-42c8-be48-eca9fc350902',
  'cd2b9d1c-a7b2-49a2-adb1-7217f2c732bf',
  'df72bce3-696f-487d-a9bc-bd30860e669e',
  'd3a64d0c-0792-4dfe-84cc-2c91fab72194',
  'fd9cfc5d-41b5-49e0-bfaf-aed5d9a9c31e',
  'da7714ef-3802-416a-b341-5eba87fb751e',
  '728b3b08-e0c5-4951-802e-e0d580cc4683',
  'f6a6c86e-bb83-4085-8be6-b8144708a190',
  'ac70ffb1-182b-4df5-9c87-a83166c21cb9',
  '49726fc1-cbfb-498b-a914-7529990501ec',
  '54abdc69-da6a-44d4-94f0-d43b173bca99',
  '21922e24-132c-49ef-93b5-43ccc137edd1',
  '3787c84c-3933-48fd-9d0c-fdf5517f9e07',
  '5eb78116-9339-448e-8f3c-72550c5dfea2',
  '0fe57d1e-ff1b-4862-9b1c-7dc27ddf3d8b',
  'a4ef7620-ff4d-49a5-b77b-bc7ef2ce2034',
  'c5b051d9-2069-41c7-acb2-541a686f6b9c',
  '202207ff-f8cc-4add-ba1d-d5a31e3dced0',
  '494b46cb-305b-4964-a8c6-091d4a12a014',
  '21ee1a5c-b218-48e8-a00a-806ec03650d4',
  '493375ba-5b45-420e-a0ba-

### Суммаризация чанков, диалога, собеседника

In [3]:
docs = []
for dialog in dialogs_sm: 
  interlocutor_summary = ""
  dialog_summary = ""
  chunk_summary = ""

  chunked_dialog =chunk_dialog(dialog['messages'], 50000)
  



In [5]:
'''
structured output
{
  "name": "dialog_summary_schema",
  "schema": {
    "type": "object",
    "properties": {
      "interlocutor_summary": {
        "type": "string",
        "description": "A summary of the interlocutor's statements."
      },
      "dialog_summary": {
        "type": "string",
        "description": "A summary of the overall dialog."
      },
      "chunk_summary": {
        "type": "string",
        "description": "A summary of a specific chunk of the dialog."
      }
    },
    "required": [
      "interlocutor_summary",
      "dialog_summary",
      "chunk_summary"
    ],
    "additionalProperties": false
  },
  "strict": true
}'''


'{\n  "name": "dialog_summary_schema",\n  "schema": {\n    "type": "object",\n    "properties": {\n      "interlocutor_summary": {\n        "type": "string",\n        "description": "A summary of the interlocutor\'s statements."\n      },\n      "dialog_summary": {\n        "type": "string",\n        "description": "A summary of the overall dialog."\n      },\n      "chunk_summary": {\n        "type": "string",\n        "description": "A summary of a specific chunk of the dialog."\n      }\n    },\n    "required": [\n      "interlocutor_summary",\n      "dialog_summary",\n      "chunk_summary"\n    ],\n    "additionalProperties": false\n  },\n  "strict": true\n}'